In [1]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd

cell="jrc_hela-2"

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(8)
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 2, min_branch_length = 100, use_skeletons=True)
lazy_results = []
for i in range(1,422,1):#range(1,87077,100): #
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
df = pd.DataFrame.from_records(results)
temp = df[["principal_inertia_component_0","principal_inertia_component_1","principal_inertia_component_2"]].to_numpy()>=0
print(temp.all())


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-c0zxu1yf', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-et0cg2dh', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-4cffrd38', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-_ehm4558', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Progr

True


In [95]:
from __future__ import print_function

import argparse
import webbrowser

import neuroglancer
import neuroglancer.cli
import copy
import requests

from IPython.display import IFrame
import numpy as np

#ap = argparse.ArgumentParser()
#neuroglancer.cli.add_server_arguments(ap)
#args = ap.parse_args()
#neuroglancer.cli.handle_server_arguments(args)


viewer = neuroglancer.Viewer()
response = requests.get(f"https://janelia-cosem-datasets.s3.amazonaws.com/{cell}/neuroglancer/mesh/mito_seg/segment_properties/info")
info_file = response.json()
segment_ids = info_file["inline"]["ids"]
mesh_id_to_index_dict = {}
with viewer.txn() as s:
    s.layers["mesh"] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
        source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg",
    )
    for index,segment_id in enumerate(segment_ids):
        mesh_id_to_index_dict[int(segment_id)] = index
        s.layers["mesh"].segments.add(segment_id)
    s.layout = '3d'

def my_action(s, class_key, color):

    if s.selected_values["mesh"].value:
        selected_mesh_ids = s.selected_values["mesh"].value
        if s.selected_values["mesh"].value:
            new_state = copy.deepcopy(viewer.state)
            new_state.layers['mesh'].segments.remove(selected_mesh_ids)
            class_layer = f'class {class_key}'
            if class_layer not in new_state.layers: #should do it with name since this is a whole object
                new_state.layers[class_layer] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
                    source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg")
            new_state.layers[class_layer].segments.add(selected_mesh_ids)

            segment_colors = {}
            for segment_id in new_state.layers[class_layer].segments:
                segment_colors[segment_id] = color
            new_state.layers[class_layer].segment_colors = segment_colors
            
            viewer.set_state(new_state)

def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id
     
def my_action_h(s):
    my_action(s,"h", "red")
def my_action_j(s):
    my_action(s,"j","gray")
def my_action_k(s):
    my_action(s,"k","blue")
def my_action_l(s):
    my_action(s,"l","magenta")

class_keys = ["h","j","k","l"]
viewer.actions.add("my-action-h", my_action_h)
viewer.actions.add("my-action-j", my_action_j)
viewer.actions.add("my-action-k", my_action_k)
viewer.actions.add("my-action-l", my_action_l)
#viewer.actions.add("my-action-p", fit_and_predict)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer["keyh"] = "my-action-h"
    s.input_event_bindings.viewer["keyj"] = "my-action-j"
    s.input_event_bindings.viewer["keyk"] = "my-action-k"
    s.input_event_bindings.viewer["keyl"] = "my-action-l"
    s.input_event_bindings.viewer["keyp"] = "my-action-p"


    s.status_messages["hello"] = "Welcome to this example"

url = viewer
display(IFrame(url, width=1200, height=800))

In [96]:
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

class FitAndPredict():
    def __init__(self, df, metrics):
        self.metrics = df[metrics].to_numpy()
    
    def fit(self, mesh_index_to_class_dict):
        self.classifier = MLPClassifier(alpha=1, max_iter=1000)
        classes = list(mesh_index_to_class_dict.values())

        mesh_indices = list(mesh_index_to_class_dict.keys())
        data = self.metrics[mesh_indices, :]
        self.scaler = preprocessing.StandardScaler().fit(data)
        data_transformed = self.scaler.transform(data)
        self.classifier.fit(data_transformed, classes)
        print(*zip(data_transformed,classes))
    def predict(self):
        test_transformed = self.scaler.transform(self.metrics)
        class_predictions = self.classifier.predict(test_transformed)
        return class_predictions

viewer.actions.remove("my-action-p",fit_and_predict)
def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id
    print(mesh_index_to_class_dict)
    
    fp = FitAndPredict(df, ["principal_inertia_component_normalized_0","principal_inertia_component_normalized_1","principal_inertia_component_normalized_2"])
    fp.fit(mesh_index_to_class_dict)
    class_predictions = fp.predict()

    new_state = copy.deepcopy(viewer.state)
    mesh_layer = new_state.layers['mesh']

    colors = ["red", "gray", "blue", "magenta"]
    
    segment_colors = {}
    for segment_id in mesh_layer.segments:
        mesh_index = mesh_id_to_index_dict[int(segment_id)]
        class_id = class_predictions[mesh_index]
        segment_colors[segment_id] = colors[class_id]

    mesh_layer.segment_colors = segment_colors
    viewer.set_state(new_state)

viewer.actions.add("my-action-p", fit_and_predict)

{370: 0, 182: 0, 344: 0, 125: 0, 286: 0, 323: 1, 4: 1, 326: 1, 296: 2, 329: 2, 365: 2}
(array([ 0.98568401, -1.25338982, -0.60167748]), 0) (array([ 1.08852282, -1.21247155, -0.87146363]), 0) (array([ 1.2500263 , -0.90156936, -1.59253787]), 0) (array([ 1.08050155, -1.02645458, -1.07855837]), 0) (array([ 0.92991289, -0.66926557, -1.18643057]), 0) (array([-0.6305076 ,  0.50471407,  0.74302184]), 1) (array([-0.6011343 ,  0.37337558,  0.83841716]), 1) (array([-0.57027189,  0.24075003,  0.93217223]), 1) (array([-1.12361549,  1.2497741 ,  0.90171213]), 2) (array([-1.19435458,  1.33570554,  0.94973947]), 2) (array([-1.2147637 ,  1.35883156,  0.96560507]), 2)
{330: 0, 370: 0, 182: 0, 344: 0, 125: 0, 286: 0, 323: 1, 4: 1, 326: 1, 296: 2, 329: 2, 365: 2}
(array([ 0.10100246, -0.44707072,  0.32582256]), 0) (array([ 1.01985383, -1.25653214, -0.65501203]), 0) (array([ 1.12721564, -1.21418446, -0.93543145]), 0) (array([ 1.29582229, -0.89242141, -1.68492587]), 0) (array([ 1.11884159, -1.02166929, -1.1